<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

**Solution author: Mateusz Doliński**

# Validation and cross-validation 

In this exercise you will implement a validation pipeline. 

At the end of the MSLE exercise you tested your model against the training and test datasets. As you should observe, there's a gap between the results. By validating your model, not only should you be able to anticipate the test time performance, but also have a method to compare different models.

Implement the basic validation method, i.e. a random split. Test it with your model from Exercise MSLE.

In [ ]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2022-10-09 20:16:01--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2022-10-09 20:16:02--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc644f0a4db0d2f717b59c6484de.dl.dropboxusercontent.com/cd/0/get/BuhR7dlXzBxdxB-0n-INwiJTlE2itDME5qslt-MuUwmxIvf2NXLaAm2orX6bJFxoiSiR0HEDO1p0picRPEBI9vQPgS4NozhT75zp1Ngt3B-b0kVOFJGN_QJoS9wWd2Uvbom3-5Vx0xFKkP6qYMNbp31v-rIZRsRxeWl8pZ-44ZQflg/file?dl=1# [following]
--2022-10-09 20:16:02--  https://uc644f0a4db0d2f717b59c6484de.dl.dropboxusercontent.com/cd/0/get/BuhR7dlXzBxdxB-0n-INwiJTlE2itDME5qslt-MuUwmxIvf2NXLaAm2orX6bJFxoiSiR0

In [ ]:
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from sklearn import preprocessing
from tqdm import tqdm

np.random.seed(357)
epsilon = 1e-7

In [ ]:
def load(name: str) -> Tuple[np.ndarray, np.array]:
    data = pd.read_csv(name)
    x = data.loc[:, data.columns != 'cena'].to_numpy()
    y = data['cena'].to_numpy()

    return x, y

In [ ]:
x_train, y_train = load('mieszkania.csv')
x_test, y_test = load('mieszkania_test.csv')

In [ ]:
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(x_train[:, 1])
x_train[:, 1] = labelencoder.transform(x_train[:, 1])
x_test[:, 1] = labelencoder.transform(x_test[:, 1])

x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

In [ ]:
#######################################################
# TODO: Implement the basic validation method,        # 
# compare MSLE on training, validation, and test sets #
#######################################################

def random_split(x: np.ndarray, percent: float = 0.8) -> Tuple[np.array, np.array]:
    idxs = np.random.permutation(len(x))

    idxs_train = idxs[: int(len(idxs) * percent)]
    idxs_val = idxs[int(len(idxs) * percent) :]

    return idxs_train, idxs_val


idxs_train, idxs_val = random_split(np.zeros(shape=[200, 6]))
assert not idxs_train in idxs_val

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  app.launch_new_instance()


To make the random split validation reliable, a huge chunk of training data may be needed. To get over this problem, one may apply cross-validaiton.

![alt-text](https://chrisjmccormick.files.wordpress.com/2013/07/10_fold_cv.png)

Let's now implement the method. Make sure that:
* number of partitions is a parameter,
* the method is not limited to `mieszkania.csv`,
* the method is not limited to one specific model.

In [ ]:
def log(x: np.ndarray) -> np.ndarray:
    return np.log(np.clip(x, epsilon, None))

In [ ]:
def msle(ys: np.array, ps: np.array) -> float:
    assert len(ys) == len(ps)
    return np.mean((log(1+ys)-log(1+ps))**2)

In [ ]:
def train(x: np.ndarray, y: np.array, alpha: int = 10000, max_iter: int = 10000) -> Tuple[np.ndarray, np.ndarray]:
    n_features = x.shape[1]

    k = 1 / n_features
    bias = np.exp(np.mean(log(1 + y)) - 1)
    weights = np.random.uniform(low=-np.sqrt(k), high=np.sqrt(k), size=[n_features, 1])

    for i in tqdm(range(max_iter)):
        preds = (x @ weights + bias).squeeze()

        grad_bias = 2 * np.mean((log(1 + y) - log(1 + preds)) * (-1 / (1 + preds)))
        grad_weights = (
            2 * np.mean(np.multiply(x, ((log(1 + y) - log(1 + preds)) * (-1 / (1 + preds))[np.newaxis, :]).T), axis=0)
        )[:, np.newaxis]

        bias -= grad_bias * alpha
        weights -= grad_weights * alpha

        pred_after_update = (x @ weights + bias).squeeze()
        loss = msle(y, pred_after_update)
        if i % 2000 == 0:
            print("loss: ", loss)

    return bias, weights

In [ ]:
####################################
# TODO: Implement cross-validation # 
####################################
def kfold(x: np.ndarray, y: np.array, n_splits: int = 5, shuffle: bool = False) -> List[float]:
    if shuffle:
        idxs_shuffled = np.random.permutation(len(x))

        x = x[idxs_shuffled]
        y = y[idxs_shuffled]

    val_losses = []

    ratio = int((1 / n_splits) * len(x))
    for k in range(n_splits):
        print("fold num: ", k)
        idxs_train_fold = [slice(0, max(ratio * k, 0)), slice(ratio * (k + 1), len(x))]
        idxs_val_fold = slice(ratio * k, ratio * (k + 1))

        # X Y X X X
        x_train_fold = np.r_[x[idxs_train_fold[0]], x[idxs_train_fold[1]]]
        y_train_fold = np.r_[y[idxs_train_fold[0]], y[idxs_train_fold[1]]]
        x_val_fold = x[idxs_val_fold]
        y_val_fold = y[idxs_val_fold]

        bias, weights = train(x_train_fold, y_train_fold)
        val_losses.append(msle(y_val_fold, (x_val_fold @ weights + bias).squeeze()))

    return val_losses


print()
kfold_loss = kfold(x_train, y_train, n_splits=3, shuffle=False)
print(f"kfold loss: {np.mean(kfold_loss)} +- {np.std(kfold_loss)}")




fold num:  0


  2%|▏         | 195/10000 [00:00<00:05, 1944.39it/s]

loss:  0.5754787664169929


 22%|██▏       | 2197/10000 [00:01<00:04, 1596.57it/s]

loss:  0.21343583484811937


...

loss:  0.08647501489216801


 84%|████████▍ | 8403/10000 [00:03<00:00, 2485.66it/s]

loss:  0.06823301732593312


100%|██████████| 10000/10000 [00:03<00:00, 2680.41it/s]

kfold loss: 0.06290906140076949 +- 0.0014037044814540986


Recall that sometimes validation may be tricky, e.g. significant class imbalance, having a small number of subjects, geographically clustered instances...

What could in theory go wrong here with random, unstratified partitions? Think about potential solutions and investigate the data in order to check whether these problems arise here.

In [ ]:
##############################
# TODO: Investigate the data #
##############################
overall = pd.Series(x_train[:, 1]).value_counts() / len(x_train)

idxs_train, _ = random_split(x_train[:, 1])
random = pd.Series(x_train[idxs_train, 1]).value_counts() / len(idxs_train)

compare = pd.DataFrame({'overall': overall, 'random': random}).sort_index()
compare['rand error %'] = 100 * random / overall - 100

print(compare)

     overall   random  rand error %
0.0    0.235  0.25000      6.382979
1.0    0.245  0.23750     -3.061224
2.0    0.270  0.26875     -0.462963
3.0    0.250  0.24375     -2.500000
